# Interact with decay data in ENDF-6 format (MF=8, MT=457)

First import `sandy` and other packages importing for formatting and postprocessing that are used in this notebook.

In [ ]:
import os
import urllib
import yaml

import pandas as pd

In [ ]:
import sandy

## Read decay data in ENDF-6 format

As always, any nuclear data evaluation can be imported into a `Endf6` instance.

Here we import the JEFF-3.3 radioactive decay data library from the [OECD/NEA website](https://www.oecd-nea.org/dbdata/jeff/jeff33/index.html).

In [ ]:
url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-rdd_all.asc"
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')

In [ ]:
tape = sandy.Endf6.from_text(text)
tape

The ENDF-6 data in `tape` can be parsed and extracted into a hierarchical format (nested `dict`) using method `DecayData.from_endf6`.

In [ ]:
rdd = sandy.DecayData.from_endf6(tape)

A better rendering of the `rdd` data content can be obtained with the `yaml` python package.

In [ ]:
print(yaml.dump(rdd.data[551340]))

In [ ]:
print(yaml.dump(rdd.data[400900]))

The description of the `rdd.data` structure is explained below, where `zam` and `zap` are 
equal to `Z * 10000 + A * 10 + M` for the parent and daughter isotopes, respectively, with 
`Z`, `A`, and `M` being the charge, nucleons and metastate numbers.

```yaml
    zam :
        half_life : value in s
        decay_constant : value in 1/s
        stable : True/False
        decay_energy :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_energy_uncertainty :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_modes :
            rtyp : 
                decay_products : 
                    zap : yield
                    zap_2 : yield_2
                    ...
                branching_ratio : value
            rtyp_2 : ...
    zam_2 : ...
```

`rtyp` is a string storing a sequence of integers, each of which defining the 
several decay events covered by a given decay mode.
The list of individual decay events is accessible via variable `sandy.decay_modes`.

In [ ]:
sandy.decay_modes

## Extract structured information into dataframes

We can extract the decay chains information (decay constant `LAMBDA` and branching ratio `YIELD`) into a `pandas.DataFrame`.

In [ ]:
chains = rdd.get_decay_chains()
chains

## Convert decay data into HDF5 format

Codes like ALEPH read radioactive decay data from nuclear data files in HDF5 format.
To produce such files we can use `sandy` and method `DecayData.to_hdf5`.

As an example, we can transfer the whole content of the `fy` instance to a HDF5 file `'decay.hdf5'`.

In [ ]:
%%capture
h5filename = "decay.hdf5"
libname = "jeff_33"
if os.path.exists(h5filename):
    os.remove(h5filename)
rdd.to_hdf5(h5filename, libname)

Decay data can also be read directly from a HDF5 file using method `DecayData.from_hdf5`.

In [ ]:
rdd2 = sandy.DecayData.from_hdf5(h5filename, libname)

In [ ]:
chains2 = rdd2.get_decay_chains()
pd.testing.assert_frame_equal(chains, chains2)